In [1]:
import sys
sys.path.append('../../includes/')
import os
from preprocessor import write_keywords,write_birth,write_parameters
from gamma import domain_mgr, heat_solve_mgr,load_toolpath,get_toolpath
import cupy as cp
import numpy as np
import pyvista as pv
import vtk

# Before starting, check using nvidia-smi
cp.cuda.Device(0).use()

<CUDA Device 0>

In [2]:
def save_vtk(filename):
    active_elements = domain.elements[domain.active_elements].tolist()
    active_cells = np.array([item for sublist in active_elements for item in [8] + sublist])
    active_cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(active_elements))
    points = domain.nodes.get()
    active_grid = pv.UnstructuredGrid(active_cells, active_cell_type, points)
    active_grid.point_data['temp'] = heat_solver.temperature.get()
    active_grid.save(filename)

In [3]:
domain = domain_mgr(filename='wall.k')
heat_solver = heat_solve_mgr(domain)

Time of reading input files: 1.3231086730957031
Time of calculating critical timestep: 0.5019655227661133
Time of reading and interpolating toolpath: 0.03844809532165527
Number of nodes: 96874
Number of elements: 83270
Number of time-steps: 142500
Time of generating surface: 1.5336432456970215


In [4]:
output_step = 0.5  # output time step
# initialization
file_num = 0
if not os.path.exists('./vtk'):
    os.mkdir('./vtk')

# save file
filename = 'vtk/u{:05d}.vtk'.format(file_num)
save_vtk(filename)
file_num = file_num + 1
output_time = domain.current_time

# time loop
while domain.current_time < 2 - 1e-8:
    # here is where you want to make changes at each time step
    heat_solver.time_integration()
    
    # save file
    if domain.current_time >= output_time + output_step:
        print("Current time:  {}, Percentage done:  {}%".format(
            domain.current_time, 100 * domain.current_time / domain.end_time))
        filename = 'vtk/u{:05d}.vtk'.format(file_num)
        save_vtk(filename)
        file_num = file_num + 1
        output_time = domain.current_time

Current time:  2.0000000000000013, Percentage done:  40.00000000000003%
Current time:  4.001999999999781, Percentage done:  80.03999999999562%


These are the parameters you might need for this project:

In [5]:
# absorpted laser power
heat_solver.q_in 

160.0

In [6]:
# nodal temperature array
heat_solver.temperature

array([300.03985522, 367.72030422, 365.4099788 , ..., 300.        ,
       300.        , 300.        ])

In [7]:
# node coordinates
domain.nodes

array([[ 22.626017,   0.892857,  -8.      ],
       [  3.533256,  -1.680824,  -2.      ],
       [  3.155609,  -1.686837,  -2.      ],
       ...,
       [-23.      ,   1.25    ,  30.      ],
       [-23.      ,  -1.25    ,  29.625   ],
       [-23.      ,  -1.25    ,  30.      ]])